<a href="https://colab.research.google.com/github/rym621jp/ImageModifier/blob/main/DHH_modifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

In [ ]:
def PrevImg(data):
    #cv2.imshow("preview", data)

    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    plt.imshow(data)
    plt.show()
    
def SaveImg(name, data):
    cv2.imwrite("E:\\DHH_modifired\\"+name.split(".")[0]+"_mod."+name.split(".")[1], data)
    
def Crop(data):   #画像を1024px四方にクロップ
    height, width, channels = data.shape
    longer = max([height, width])
    racio = 1024/longer
    
    result = cv2.resize(data, dsize=None, fx=racio, fy=racio)
    
    return result

def AddImage(back, main):   #backの画像にmainの画像を上書きする
    height, width, _ = main.shape
    x_offset = 0
    y_offset = 0
    
    if height < width:
        y_offset = int((1024-height)/2)
    elif width < height:
        x_offset = int((1024-width)/2)
        
    back[y_offset:y_offset+height, x_offset:x_offset+width] = main
    
    return back

def AnalyzeDist(data):   #四辺のカラーデータを取得
    height, width, _ = data.shape
    #edge_cols = data[0, 0:width-1].reshape(1, width-1, 3) + data[0:height-1, width-1].reshape(height-1, 1, 3) + data[height-1, 0:width-1].reshape(1, width-1, 3) + data[0:height-1, 0].reshape(height-1, 1, 3)
    edge_cols = []
    for col in data[0, 0:width-1]:
        edge_cols.append(col.tolist())
    for col in data[height-1, 0:width-1]:
        edge_cols.append(col.tolist())
    for col in data[0:height-1, 0]:
        edge_cols.append(col.tolist())
    for col in data[0:height-1, width-1]:
        edge_cols.append(col.tolist())
    return edge_cols

def ColAve(img):   #受け取った画像の色平均をとる
    
    def Img2Cols(img):
        result = []
        for line in img:
            for pixel in line:
                result.append(pixel)
            
        return result

    R = []
    G = []
    B = []
    cnt = 0
    for cols in img:
        R.append(cols[0])
        G.append(cols[1])
        B.append(cols[2])
    return (int(np.mean(R)), int(np.mean(G)), int(np.mean(B)))

def ColAdjust(img):   #RGBの順番を修正
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

In [ ]:
origin = "E:\\DHH_archive_copy\\"   #元画像フォルダ
fixed = "E:\\DHH_modifired\\"   #保存先フォルダ

In [ ]:
#対象画像をfilesに格納
files = glob.glob(origin+"*.*")
files

In [ ]:
for path in files:
    #画像読み込み&サイズの正規化
    img = Crop(cv2.imread(path))
    #ファイル名取得
    name = path.split("\\")[2]
 
    if img.shape[:2]==(1024,1024):   #画像が正方形ならそのまま出力
        cv2.imwrite(fixed+name.split(".")[0]+".jpg", img, [cv2.IMWRITE_JPEG_QUALITY, 100])
    else:   #それ以外なら画像の四辺の色に調和した色で余白を作って正方形にする
        bg_col = ColAve(AnalyzeDist(img))
        bg = np.full((1024, 1024, 3), bg_col)
        result_img = AddImage(bg, img)
        cv2.imwrite(fixed+name.split(".")[0]+".jpg", result_img, [cv2.IMWRITE_JPEG_QUALITY, 100])